    Reinforced learning can be explained by giving the following simple example: Consider you own a puppy. Since animals don't understand our language you can'y tell them what to do. Instead, we will try to implement a reward per desired response strategy. If the puppy's response is the desired one, we reward him with a snack. 

    In a way, Reinforcement Learning is the science of making optimal decisions using experiences. Breaking it down, the process of Reinforcement Learning involves these simple steps:

Observation of the environment
Deciding how to act using some strategy
Acting accordingly
Receiving a reward or penalty
Learning from the experiences and refining our strategy
Iterate until an optimal strategy is found

Reference: https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

Firs of all we will need to install Gym, whcih provides us already built environments where we can plug in our code and test the agent

In [3]:

!pip install cmake 'gym[atari]' scipy


     |████████████████████████████████| 42.8 MB 886 kB/s  eta 0:00:01
     |████████████████████████████████| 1.6 MB 43.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 28.5 MB/s eta 0:00:01
     |████████████████████████████████| 40.3 MB 54.4 MB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.18.0-py3-none-any.whl size=1656448 sha256=66dcde96e81d12c95e79332a3792aae005c0c631e42b4248114b13ae23b301fb
  Stored in directory: /Users/CTR/Library/Caches/pip/wheels/99/f7/e3/d6f0f120ac047c1e5de2ae34930e7bf6e8de1c7a4d5fa68555
Successfully built gym


Now that it is installed we can load our environment with the following command

In [5]:
import gym

env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



    We will use the following methods in our project
    
    env.reset: Resets the environment and returns a random initial state.
    env.render: Renders one frame of the environment (helpful in visualizing the environment)
    env.step(action): Step the environment by one timestep. Returns
        observation: Observations of the environment
        reward: If your action was beneficial or not
        done: Indicates if we have successfully picked up and dropped off a passenger, also called one episode
        info: Additional info such as performance and latency for debugging purposes
    

In [8]:
env.reset() 
env.render()

print(f"Action Space {env.action_space}")
print(f"State Space {env.observation_space}")

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


The filled yellow square here represents a taxi, which is full where the green one represents its being empty.
The pipe ("|") represents a wall which the taxi cannot cross. As verified by the prints, we have an Action Space of size 6 and a State Space of size 500. Our RL algorithm won't need more than these two things. All we need now is to assign action numbers from 0-5 for south,north, east, west, drop off and pick up. 

We can now take our illustration above, encode its state, and give it to the environment to render in Gym.

In [9]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Reward Table
When the environment was created, the initial reward table "P" also was created. We can think of it as a matrix of states x rows. 

In [10]:
env.P[328]


{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

These are the default reward values assigned to our illustration's state

## Methods

In this notebook I'll be using q-learning method to tackle the problem.(although it is possible to solve it brute-force)

Essentially, Q-learning lets the agent use the environment's rewards to learn, over time, the best action to take in a given state.

In our Taxi environment, we have the reward table, P, that the agent will learn from. It does thing by looking receiving a reward for taking an action in the current state, then updating a Q-value to remember if that action was beneficial.

The values store in the Q-table are called a Q-values, and they map to a (state, action) combination.

Q(state,action)←(1−α)Q(state,action)+α(reward+γmaxaQ(next state,all actions)), where α is learning rate and γ is discount factor


## Q Learning Break down Process

1)Initialize the Q-table by all zeros.

2)Start exploring actions: For each state, select any one among all possible actions for the current state (S).

3)Travel to the next state (S') as a result of that action (a).

4)For all possible actions from the state (S') select the one with the highest Q-value.

5)Update Q-table values using the equation.

6)Set the next state as the current state.

7)If goal state is reached, then end and repeat the process.


500×6 matrix of zeros

In [12]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [13]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done: # we decide whether to pick a random action or to exploit the already computed Q-values.
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 48.1 s, sys: 15.3 s, total: 1min 3s
Wall time: 51.7 s


Now let's check what are our Q values

In [14]:
q_table[328]


array([ -2.39709366,  -2.27325184,  -2.40449351,  -2.35954514,
       -11.18632961, -10.95260753])

Evaluating agent's performance after Q-learning

In [15]:
total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.23
Average penalties per episode: 0.0


Reference:
State Abstraction for Programmable Reinforcement Learning Agents by David Andre
&&
https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/